# Stochastic Simulations Part 2 - Repressilator

This notebook strongly relies on the basis of a tutorial from the 'Design Principles of Genetic Circuits' course at Caltech, Spring term, 2019 (http://be150.caltech.edu/2019/) by Bois and Elowitz (see references of thesis). In particular, the choice of the Python packages was determined by the course. Text passages & code blocks from the templates that were provided as a part of the course were copied and modified. 

In [1]:
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import numba

import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting

from bokeh.io import export_png

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
%load_ext line_profiler

Loading BokehJS ...

## The stochastic model for the toggle_switch

For the Gillespie simulations we need to specify the propensities and updates, along with an initial population. 

The model takes the follwowing form:
We explicitly consider both mRNA and protein. A repressor binds the operator with chemical rate constant $k_r$, and an operator may have zero, one, or two repressors bound. The unbinding rate of a repressor when one is bound is $k_{u,1}$ and that of a repressor when two are bound if $k_{u,2}$, with $k_{u,2} < k_{u,1}$ to capture cooperativity. If no repressors are bound to a promoter region transciption happens at rate constant $k_{mu}$. If one or two repressors are bound, basal transcription can occur, and transciption happens at rate constant $k_{mo}$.

This is the table of the populations we are keeping track of:

|index|description | variable|
|:----:|:------ |:------: |
|0|gene 1 mRNA copy number | `m1`|
|1|gene 1 protein copy number | `p1`|
|2|gene 2 mRNA copy number | `m2`|
|3|gene 2 protein copy number | `p2`|
|4|gene 3 mRNA copy number | `m3`|
|5|gene 3 protein copy number | `p3`|
|6|Number of repressors bound to promoter of gene 3| `n1`|
|7|Number of repressors bound to promoter of gene 1| `n2`|
|8|Number of repressors bound to promoter of gene 2| `n3`|

Note that we labeled each species with an index, which corresponds to its position in the array of populations in the simulation.

Next, we can set up a table of updates and propensities for the moves we allow in the Gillespie simulation. We also assign an index to each entry here, as this helps keep track of everything.

|index|description | update | propensity|
|:----|:------ |:------ | :----:|
|0|transcription of gene 1 mRNA| `m1 ⟶ m1 + 1`| `kmu*(n3 == 0) + kmo*(n3 > 0)`|
|1|transcription of gene 2 mRNA| `m2 ⟶ m2 + 1`| `kmu*(n1 == 0) + kmo*(n1 > 0)`|
|2|transcription of gene 3 mRNA| `m3 ⟶ m3 + 1`| `kmu*(n2 == 0) + kmo*(n2 > 0)`|
|3|translation of gene 1 protein| `p1 ⟶ p1 + 1`| `kp * m1`|
|4|translation of gene 1 protein| `p2 ⟶ p2 + 1`| `kp * m2`|
|5|translation of gene 1 protein| `p3 ⟶ p3 + 1`| `kp * m3`|
|6|degradation of gene 1 mRNA| `m1 ⟶ m1 - 1`| `gamma_m * m1`|
|7|degradation of gene 2 mRNA| `m2 ⟶ m2 - 1`| `gamma_m * m2`|
|8|degradation of gene 3 mRNA| `m3 ⟶ m3 - 1`| `gamma_m * m3`|
|9|degradation of unbound gene 1 protein| `p1 ⟶ p1 - 1` | `gamma_p * p1`|
|10|degradation of unbound gene 2 protein| `p2 ⟶ p2 - 1` | `gamma_p * p2`|
|11|degradation of unbound gene 3 protein| `p3 ⟶ p3 - 1` | `gamma_p * p3`|
|12|degradation of bound gene 1 protein| `n1 ⟶ n1 - 1` | `gamma_p * n1`|
|13|degradation of bound gene 2 protein| `n2 ⟶ n2 - 1` | `gamma_p * n2`|
|14|degradation of bound gene 3 protein| `n3 ⟶ n3 - 1` | `gamma_p * n3`|
|15|binding of protein to gene 1 operator| `n3 ⟶ n3 + 1`, `p3 ⟶ p3 - 1`| `kr * p3 * (n3 < 2)`|
|16|binding of protein to gene 2 operator| `n1 ⟶ n1 + 1`, `p1 ⟶ p1 - 1`| `kr * p1 * (n1 < 2)`|
|17|binding of protein to gene 3 operator| `n2 ⟶ n2 + 1`, `p2 ⟶ p2 - 1`| `kr * p2 * (n2 < 2)`|
|18|unbinding of protein to gene 1 operator| `n3 ⟶ n3 - 1`, `p3 ⟶ p3 + 1`| `ku1*(n3 == 1) + 2*ku2*(n3 == 2)`|
|19|unbinding of protein to gene 2 operator| `n1 ⟶ n1 - 1`, `p1 ⟶ p1 + 1`| `ku1*(n1 == 1) + 2*ku2*(n1 == 2)`|
|20|unbinding of protein to gene 3 operator| `n2 ⟶ n2 - 1`, `p2 ⟶ p2 + 1`| `ku1*(n2 == 1) + 2*ku2*(n2 == 2)`|

Finally, we have parameters that were introduced in the propensities, so we should have a table defining them.


|parameter| value | units |
|:----|:------ |:------: |
|`kmu`| 0.5 | 1/sec|
|`kmo`|0.00001  | 1/sec|
|`kp`| 0.167 | 1/molec-sec|
|`gamma_m`| 0.05776 | 1/sec|
|`gamma_p`| 0.00155 | 1/sec|
| `kr` | 1.0 | 1/molec-sec|
| `ku1` | 5.0 | 1/sec|
| `ku2` | 0.05 | 1/sec|

In [2]:
@numba.njit
def repressilator_propensity(propensities, population, t, 
                             kmu,
                             kmo,
                             kp,
                             gamma_m,
                             gamma_p,
                             kr,
                             ku1,
                             ku2):
    m1, p1, m2, p2, m3, p3, n1, n2, n3 = population
    
    propensities[0] = kmu if n3 == 0 else kmo
    propensities[1] = kmu if n1 == 0 else kmo
    propensities[2] = kmu if n2 == 0 else kmo
    propensities[3] = kp * m1                
    propensities[4] = kp * m2                
    propensities[5] = kp * m3                
    propensities[6] = gamma_m * m1           
    propensities[7] = gamma_m * m2           
    propensities[8] = gamma_m * m3           
    propensities[9] = gamma_p * p1           
    propensities[10] = gamma_p * p2          
    propensities[11] = gamma_p * p3          
    propensities[12] = gamma_p * n1          
    propensities[13] = gamma_p * n2          
    propensities[14] = gamma_p * n3          
    propensities[15] = kr * p3 * (n3 < 2)    
    propensities[16] = kr * p1 * (n1 < 2)    
    propensities[17] = kr * p2 * (n2 < 2)    
    propensities[18] = ku1*(n3==1) + 2*ku2*(n3==2)
    propensities[19] = ku1*(n1==1) + 2*ku2*(n1==2)
    propensities[20] = ku1*(n2==1) + 2*ku2*(n2==2)

In [3]:
repressilator_update = np.array([
    # 0   1   2   3   4   5   6   7   8
    [ 1,  0,  0,  0,  0,  0,  0,  0,  0], # 0
    [ 0,  0,  1,  0,  0,  0,  0,  0,  0], # 1
    [ 0,  0,  0,  0,  1,  0,  0,  0,  0], # 2
    [ 0,  1,  0,  0,  0,  0,  0,  0,  0], # 3
    [ 0,  0,  0,  1,  0,  0,  0,  0,  0], # 4
    [ 0,  0,  0,  0,  0,  1,  0,  0,  0], # 5
    [-1,  0,  0,  0,  0,  0,  0,  0,  0], # 6
    [ 0,  0, -1,  0,  0,  0,  0,  0,  0], # 7
    [ 0,  0,  0,  0, -1,  0,  0,  0,  0], # 8
    [ 0, -1,  0,  0,  0,  0,  0,  0,  0], # 9
    [ 0,  0,  0, -1,  0,  0,  0,  0,  0], # 10
    [ 0,  0,  0,  0,  0, -1,  0,  0,  0], # 11
    [ 0,  0,  0,  0,  0,  0, -1,  0,  0], # 12
    [ 0,  0,  0,  0,  0,  0,  0, -1,  0], # 13
    [ 0,  0,  0,  0,  0,  0,  0,  0, -1], # 14
    [ 0,  0,  0,  0,  0, -1,  0,  0,  1], # 15
    [ 0, -1,  0,  0,  0,  0,  1,  0,  0], # 16
    [ 0,  0,  0, -1,  0,  0,  0,  1,  0], # 17
    [ 0,  0,  0,  0,  0,  1,  0,  0, -1], # 18
    [ 0,  1,  0,  0,  0,  0, -1,  0,  0], # 19
    [ 0,  0,  0,  1,  0,  0,  0, -1,  0], # 20
    ], dtype=int)

## Basic Form of the Repressilator

In [4]:
# Parameter values
kmu = 0.5
kmo = 0.00001
kp = 0.167
gamma_m = 0.05776
gamma_p = 0.00155
kr = 1.0
ku1 = 5.0
ku2 = 0.05

repressilator_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

In [5]:
# State with 10 copies of everything, nothing bound to operators
repressilator_pop_0 = np.array([10, 10, 10, 10, 10, 10, 0, 0, 0], dtype=int)

repressilator_time_points = np.linspace(0, 80000, 4001)

In [6]:
# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(repressilator_propensity, 
                                repressilator_update, 
                                repressilator_pop_0, 
                                repressilator_time_points, 
                                args=repressilator_args)


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')
for c, i in enumerate([1, 3, 5]):
    p.line(repressilator_time_points/60, 
           pop[0,:,i], color=colors[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

export_png(p, filename="../Results/Simulations/Repressilator_Basic_Form.png")

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Repressilator_Basic_Form.png'

## Varying levels of leakage

kmo = 0.00001 / 0.0001 / 0.001 / 0.01

In [7]:
# Parameter values
kmu = 0.5
kmo = 0.00001
kp = 0.167
gamma_m = 0.05776
gamma_p = 0.00155
kr = 1.0
ku1 = 5.0
ku2 = 0.05

# State with 1000 copies of everything, nothing bound to operators
repressilator_pop_0 = np.array([10, 10, 10, 10, 10, 10, 0, 0, 0], dtype=int)

In [8]:
kmo = 0.00001

repressilator_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 1

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(repressilator_propensity, 
                                repressilator_update, 
                                repressilator_pop_0, 
                                repressilator_time_points, 
                                args=repressilator_args,
                                size = no_of_runs)
                


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')

for jj in range(no_of_runs):
    for c, i in enumerate([1, 3, 5]):
        p.line(repressilator_time_points/60, 
               pop[jj,:,i], color=colors[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Repressilator_Leakage" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Repressilator_Leakage1e-05.png'

In [9]:
kmo = 0.0001

repressilator_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 1

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(repressilator_propensity, 
                                repressilator_update, 
                                repressilator_pop_0, 
                                repressilator_time_points, 
                                args=repressilator_args,
                                size = no_of_runs)
                


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')

for jj in range(no_of_runs):
    for c, i in enumerate([1, 3, 5]):
        p.line(repressilator_time_points/60, 
               pop[jj,:,i], color=colors[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Repressilator_Leakage" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Repressilator_Leakage0.0001.png'

In [10]:
kmo = 0.001

repressilator_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 1

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(repressilator_propensity, 
                                repressilator_update, 
                                repressilator_pop_0, 
                                repressilator_time_points, 
                                args=repressilator_args,
                                size = no_of_runs)
                


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')

for jj in range(no_of_runs):
    for c, i in enumerate([1, 3, 5]):
        p.line(repressilator_time_points/60, 
               pop[jj,:,i], color=colors[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Repressilator_Leakage" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Repressilator_Leakage0.001.png'

In [11]:
kmo = 0.005

repressilator_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 1

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(repressilator_propensity, 
                                repressilator_update, 
                                repressilator_pop_0, 
                                repressilator_time_points, 
                                args=repressilator_args,
                                size = no_of_runs)
                


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')

for jj in range(no_of_runs):
    for c, i in enumerate([1, 3, 5]):
        p.line(repressilator_time_points/60, 
               pop[jj,:,i], color=colors[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Repressilator_Leakage" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Repressilator_Leakage0.005.png'

In [12]:
kmo = 0.01

repressilator_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 1

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(repressilator_propensity, 
                                repressilator_update, 
                                repressilator_pop_0, 
                                repressilator_time_points, 
                                args=repressilator_args,
                                size = no_of_runs)
                


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')

for jj in range(no_of_runs):
    for c, i in enumerate([1, 3, 5]):
        p.line(repressilator_time_points/60, 
               pop[jj,:,i], color=colors[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Repressilator_Leakage" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Repressilator_Leakage0.01.png'

In [13]:
kmo = 0.05

repressilator_args = (kmu,
                      kmo,
                      kp,
                      gamma_m,
                      gamma_p,
                      kr,
                      ku1,
                      ku2)

no_of_runs  = 1

# Perform the Gillespie simulation
pop = biocircuits.gillespie_ssa(repressilator_propensity, 
                                repressilator_update, 
                                repressilator_pop_0, 
                                repressilator_time_points, 
                                args=repressilator_args,
                                size = no_of_runs)
                


# Make plot
colors = bokeh.palettes.d3['Category10'][3]
p = bokeh.plotting.figure(height=250, width=600, 
                          x_axis_label='time (min)',
                          y_axis_label='protein copy number')

for jj in range(no_of_runs):
    for c, i in enumerate([1, 3, 5]):
        p.line(repressilator_time_points/60, 
               pop[jj,:,i], color=colors[c])

bokeh.io.show(p)

p.toolbar.logo = None
p.toolbar_location = None

png_name = "../Results/Simulations/Repressilator_Leakage" + str(kmo) + ".png"
export_png(p, filename = png_name)

'/Users/Gordian/Documents/GitHub/MT4599/Results/Simulations/Repressilator_Leakage0.05.png'